<a href="https://colab.research.google.com/github/Avinashabilash/rag-chunk/blob/main/RAG--.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pdfplumber sentence-transformers faiss-cpu numpy scikit-learn tqdm PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 70.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 56.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 90.0 MB/s eta 0:00:00


In [ ]:
import os
import re

In [ ]:
import PyPDF2

In [ ]:
import numpy as np

In [ ]:
import pdfplumber

In [ ]:
import faiss

In [ ]:
from termcolor import colored


In [ ]:
from tqdm import tqdm

In [ ]:
pip install tf-keras

In [ ]:
pip install "keras<3"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.10.0
    Uninstalling keras-3.10.0:
      Successfully uninstalled keras-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires keras>=3.5.0, but you have keras 2.15.0 which is incompatible.
keras-hub 0.21.1 requires keras>=3.5, but you have keras 2.15.0 which is incompatible.


In [ ]:
from sentence_transformers import SentenceTransformer

In [ ]:
"HF_API_KEY"] = "ENTER YOUT API"

# Optional: login via huggingface_hub
from huggingface_hub import login
login(token=["HF_API_KEY"])

In [ ]:
def load_pdf_text(path: str):
    pages = []
    with pdfplumber.open(path) as pdf:
        for p in pdf.pages:
            text = p.extract_text() or ""
            pages.append(text)
    full = "\n\n".join(pages)
    return full, pages

def get_pdf_metadata(path: str):
    try:
        reader = PyPDF2.PdfReader(path)
        meta = reader.metadata or {}
        return {k.strip('/'): str(v) for k,v in meta.items() if v}
    except:
        return {}


In [ ]:
def chunk_text(text, chunk_size=500, overlap=100):
    chunks = []
    start = 0
    while start < len(text):
        end = min(start + chunk_size, len(text))
        chunk = text[start:end].strip()
        if chunk:
            chunks.append(chunk)
        start = end - overlap
        if start < 0:
            start = 0
    return chunks

In [ ]:
class RagStore:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        print("Loading embedding model:", model_name)
        self.model = SentenceTransformer(model_name)
        self.index = None
        self.chunks = []
        self.metadata = []

    def _normalize(self, x):
        norms = np.linalg.norm(x, axis=1, keepdims=True)
        norms[norms==0] = 1
        return x / norms

    def embed_chunks(self, chunks):
        embs = self.model.encode(chunks, convert_to_numpy=True, show_progress_bar=True)
        return self._normalize(embs)

    def build_index(self, chunks):
        self.chunks = chunks
        embs = self.embed_chunks(chunks)
        dim = embs.shape[1]
        self.index = faiss.IndexFlatIP(dim)
        self.index.add(embs)
        print("FAISS index built with", self.index.ntotal, "chunks.")

    def query(self, q, top_k=3):
        q_emb = self.model.encode([q], convert_to_numpy=True)
        q_emb = self._normalize(q_emb)
        D, I = self.index.search(q_emb, top_k)
        return [(int(idx), float(score)) for idx, score in zip(I[0], D[0]) if idx != -1]

In [ ]:
from google.colab import files

# Upload the PDF file manually
uploaded = files.upload()

# The file will be saved with its name in the current directory
pdf_path = "lekl101.pdf"   # make sure this matches your uploaded file name


Saving lekl101.pdf to lekl101 (6).pdf


In [ ]:
def chunk_text(text, chunk_size=500, overlap=100):
    """
    Splits text into overlapping chunks.

    Args:
        text (str): Input text.
        chunk_size (int): Size of each chunk (characters).
        overlap (int): Number of overlapping characters between chunks.

    Returns:
        List[str]: List of text chunks.
    """
    chunks = []
    start = 0
    while start < len(text):
        end = start + chunk_size
        chunks.append(text[start:end])
        start += chunk_size - overlap
    return chunks


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss

# Upload file
uploaded = files.upload()
pdf_path = "lekl101.pdf"

# Load text
full_text, pages = load_pdf_text(pdf_path)
meta = get_pdf_metadata(pdf_path)

# Chunk per page
all_chunks = []
for i, p in enumerate(pages):
    for c in chunk_text(p, chunk_size=500, overlap=100):
        all_chunks.append({"text": c, "page": i})

# Build RAG index
store = RagStore()
store.build_index([c["text"] for c in all_chunks])


Saving lekl101.pdf to lekl101 (13).pdf
Loading embedding model: all-MiniLM-L6-v2


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

FAISS index built with 56 chunks.


In [ ]:
full_text, pages = load_pdf_text(pdf_path)


In [ ]:
print(len(pages))

11


In [ ]:
def answer_query(store, chunks, query, k=3, threshold=0.25):
    results = store.query(query, top_k=k)
    sources = [(chunks[idx]["text"], score, chunks[idx]["page"]) for idx, score in results]

    if sources and sources[0][1] >= threshold:
        return {"type": "answer", "answer": sources[0][0], "sources": sources}
    else:
        return {"type": "related", "answer": None, "sources": sources}

In [ ]:
def pretty_print_answer(query, result):
    print(colored("\n? Your question:", "red"), colored(query, "green", attrs=["underline"]))
    print(colored("🔎 Searching and generating answer...", "blue"))

    if result['type'] == 'answer':
        print(colored(f"\nFound {len(result['sources'])} relevant chunks:", "blue"))
        for i, (txt, score, page) in enumerate(result['sources'], 1):
            print(f"Chunk {i}: Score {score:.3f} (≈Page {page+1})")
        print(colored("\nAnswer:", "red"), result['answer'][:400], "...")
    else:
        print(colored(f"\nNo confident single answer. Showing top {len(result['sources'])} related chunks:", "yellow"))
        for i, (txt, score, page) in enumerate(result['sources'], 1):
            print(f"Chunk {i}: Score {score:.3f} (≈Page {page+1})")
            print("Excerpt:", txt[:200].replace("\n", " "), "...")

In [ ]:
def interactive_rag(store, chunks, top_k=3):
    while True:
        query = input("Ask a question (or type 'exit' to quit): ")
        if query.lower() == "exit":
            break
        results = store.query(query, top_k=top_k)
        for i, res in enumerate(results, 1):
            print(f"\nResult {i}:")
            print(res)


In [ ]:
def interactive_rag(store, chunks):
    print(colored("="*80, "cyan"))
    print(colored("🤖 RAG System Ready! Ask me questions about your PDF", "cyan"))
    print(colored("👉 Type 'bye', 'quit', 'exit', or 'q' to exit", "yellow"))
    print(colored("👉 Type 'info' to see database statistics", "yellow"))
    print(colored("="*80, "cyan"))

    while True:
        query = input(colored("\n? Your question: ", "red"))
        if query.lower().strip() in ["bye", "quit", "exit", "q"]:
            print(colored("Goodbye! 👋", "cyan"))
            break
        if query.lower().strip() == "info":
            print(f"Database Loaded: {len(chunks)} chunks from {len(pages)} pages")
            continue
        result = answer_query(store, chunks, query, k=3)
        pretty_print_answer(query, result)



In [ ]:
pip install termcolor sentence-transformers faiss-cpu -q

In [ ]:

from termcolor import colored
import pdfplumber
import numpy as np

# Upload file
uploaded = files.upload()
pdf_path = "lekl101.pdf"

# Load text
full_text, pages = load_pdf_text(pdf_path)
meta = get_pdf_metadata(pdf_path)

# Chunk
all_chunks = []
for i, p in enumerate(pages):
    for c in chunk_text(p, chunk_size=500, overlap=100):
        all_chunks.append({"text": c, "page": i})

# Build RAG index
store = RagStore()
store.build_index([c["text"] for c in all_chunks])

# Query
interactive_rag(store, all_chunks)


Saving lekl101.pdf to lekl101 (16).pdf
Loading embedding model: all-MiniLM-L6-v2


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

FAISS index built with 56 chunks.
🤖 RAG System Ready! Ask me questions about your PDF
👉 Type 'bye', 'quit', 'exit', or 'q' to exit
👉 Type 'info' to see database statistics

? Your question: stop and exit

? Your question: stop and exit
🔎 Searching and generating answer...

Found 3 relevant chunks:
Chunk 1: Score 0.344 (≈Page 10)
Chunk 2: Score 0.225 (≈Page 10)
Chunk 3: Score 0.224 (≈Page 6)

Answer:  is its end; the next most
emphatic is its beginning; and the least emphatic, its middle.
In the sentence above the most important fact is that the huge
wave embedded one of the cars in one side of the hotel.
The other details of time and place are given at the beginning.
The general statement of the ‘huge wave picking up several
cars’ precedes the particular car which is pertinent to the theme
of ...

? Your question: stop and think

? Your question: stop and think
🔎 Searching and generating answer...

Found 3 relevant chunks:
Chunk 1: Score 0.317 (≈Page 6)
Chunk 2: Score 0.265 (≈Page 10)
